# Time Series Prediction


In [1]:
# @ Lukman copyright 
# MIT Licence

In [44]:
# for data frame analysis
import pandas as pd 

# for mathematical operations
import numpy as np 


# matplotlib library for plotting
import matplotlib.pyplot as plt
%matplotlib inline


# for use of tensorflow
import tensorflow as tf
#from tensorflow.nn.rnn import *
from tensorflow.python.ops  import *


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
# THE MODEL IS FOR EDUCATION PURPOSES ONLY

In [2]:
# enable eager executinon
tf.enable_eager_execution()

In [3]:
# verify version
tf.__version__

'1.15.0'

# Data Preprocessing 

In [4]:
# load data set
illustration = pd.read_csv('illustration.csv')

In [5]:
# view data set
# AIM IS TO PREDICT METER READING 
illustration[1:2] 

,meter_reading,square_feet,air_temperature,dew_temperature,month,day,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,...,primary_use_Utility,site_id_2,site_id_6,site_id_7,site_id_9,site_id_10,site_id_11,site_id_13,site_id_14,site_id_15
1,-138.151378,21958,-10.64683,-0.056432,1,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [6]:
illustration.isnull().any() 


meter_reading                                False
square_feet                                  False
air_temperature                              False
dew_temperature                              False
month                                        False
day                                          False
primary_use_Education                        False
primary_use_Entertainment/public assembly    False
primary_use_Food sales and service           False
primary_use_Healthcare                       False
primary_use_Lodging/residential              False
primary_use_Manufacturing/industrial         False
primary_use_Office                           False
primary_use_Other                            False
primary_use_Parking                          False
primary_use_Public services                  False
primary_use_Religious worship                False
primary_use_Retail                           False
primary_use_Technology/science               False
primary_use_Utility            

In [7]:
# save the column names 
col_names = list(illustration.columns)

In [8]:
# the column names will be changed such that
# month_daz becomes a unit feature
# primary use becomes a unit feature
# site id becomes also a unit feature using ths Dataset API later
# in the note book
col_names

['meter_reading',
 'square_feet',
 'air_temperature',
 'dew_temperature',
 'month',
 'day',
 'primary_use_Education',
 'primary_use_Entertainment/public assembly',
 'primary_use_Food sales and service',
 'primary_use_Healthcare',
 'primary_use_Lodging/residential',
 'primary_use_Manufacturing/industrial',
 'primary_use_Office',
 'primary_use_Other',
 'primary_use_Parking',
 'primary_use_Public services',
 'primary_use_Religious worship',
 'primary_use_Retail',
 'primary_use_Technology/science',
 'primary_use_Utility',
 'site_id_2',
 'site_id_6',
 'site_id_7',
 'site_id_9',
 'site_id_10',
 'site_id_11',
 'site_id_13',
 'site_id_14',
 'site_id_15']

In [9]:
# check the data set dtypes
illustration.dtypes

meter_reading                                float64
square_feet                                    int64
air_temperature                              float64
dew_temperature                              float64
month                                          int64
day                                            int64
primary_use_Education                          int64
primary_use_Entertainment/public assembly      int64
primary_use_Food sales and service             int64
primary_use_Healthcare                         int64
primary_use_Lodging/residential                int64
primary_use_Manufacturing/industrial           int64
primary_use_Office                             int64
primary_use_Other                              int64
primary_use_Parking                            int64
primary_use_Public services                    int64
primary_use_Religious worship                  int64
primary_use_Retail                             int64
primary_use_Technology/science                

In [10]:
# save the data types of the data set 
# this is important for converting the data set into tensorflow equivalent data set
# note the data types for the incoming data should be carefully converted to
# it tensorflow type to avoid errors
data_types = list(illustration.dtypes)

In [11]:
# convert the data types to 
# tensorflow data types 
for i,j in enumerate(data_types):
    j = str(j) # convertr from numpy dtype to str
    #print(j)
    if j == 'float64':
        data_types[i] = tf.float64
    else:
        data_types[i]  = tf.int32

In [12]:
# the data types are now in tensrfloe data types
data_types[0:4]

[tf.float64, tf.int32, tf.float64, tf.float64]

### Separate train and test data 

In [13]:
# separate test train
partition= 1000
illustration_test = illustration.head(partition).copy(deep=True)
illustration_train = illustration[partition:].copy(deep=True)

In [14]:
# convert to csv
# note the index set to false and we dont need the header
# it is this new csv that goes into the pipeline
illustration_test.to_csv('illustration_test.csv',index=False,header=None)
illustration_train.to_csv('illustration_train.csv',index=False,header=None)


#### Work with downloaded csv to preserve data type
Load the csv file and parse the data types of the datatset together into the
tensorflow load pipeline

This is as demonstrated in the video

In [15]:
# load the data using the tensorflow Dataset API
# add the data types too
# the data types are tensorflow data types (see cell above)
csvData = tf.data.experimental.CsvDataset('illustration_train.csv', data_types, header=False)

In [16]:
# check the data type :
print(list(csvData.take(1)))



[(<tf.Tensor: id=46, shape=(), dtype=float64, numpy=-223.16063515656919>, <tf.Tensor: id=47, shape=(), dtype=int32, numpy=128536>, <tf.Tensor: id=48, shape=(), dtype=float64, numpy=-0.6938878093609953>, <tf.Tensor: id=49, shape=(), dtype=float64, numpy=0.3053940754239157>, <tf.Tensor: id=50, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=51, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=52, shape=(), dtype=int32, numpy=1>, <tf.Tensor: id=53, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=54, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=55, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=56, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=57, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=58, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=59, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=60, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=61, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=62, shape=(), dtype=int32, numpy=0>, <tf.Tensor: id=63, shape=(), dtype=int32, numpy=0>

#### Parse data 

In [17]:
# Parsing the data and
# merging the columns:
# ---- day and month as single column
# ---- merge the site id as a single feature vector vector
# ---- primary use as a single feature vector

# take the forst 4 col names that meter reading, square feet, air temp , dew temp,
# add it to a the new column names that would be formed after single column merging



# define new column names from the previous data set col_names
# the new col names will be used in the tensorflow API
# col_names = meter reading ,col_names[1]=sqaure feet,col_names[2] = air temperature
# col_names[3] = dew temperature
col_names_= [col_names[0],col_names[1],col_names[2],col_names[3],'month_day','primary_use','site_id' ]

def _parse_csv_row(*vals):
    '''
    Uses Feature columns
    Does feature engineering
    '''
    
    # month and day and single feature
    # the 4:6 implies take column 4 and 5
    # which corresponds to month and day
    month_day = tf.convert_to_tensor(vals[4:6])
    # primary use as single feature
    # the 6:20 implies take column 6 to 19
    # this is where the primary use is
    primary_use = tf.convert_to_tensor(vals[6:20])
    # site id as single feature
    # the 20:29 implies take column 20 to 29
    # this is the site id
    site_id =  tf.convert_to_tensor(vals[20:29])
    
    # merge the features together, note meter reading is the fist column
    # so it is excluded -- meaning index starts from 1
    # 1 to 4 are sqaure feet, air temp, dew temperature respectively
    # the others are as defined above
    feature_vals = vals[1:4] + (month_day,primary_use,site_id)
    
    
    # dictionaryof col names and  feature sets pair
    features = dict(zip(col_names_[1:],feature_vals))
    
    # name the targets or lables
    # meter reading is the tarhet
    targets_tensor = tf.convert_to_tensor(vals[0],name=col_names_[0]) 
    
    
    
    return features, targets_tensor

##### Batch the datatset

In [18]:
# map the parser to the data set we loaded for training
# csvData = loaded data above, _parse_csv_row = function above
# feel free to change batch size
dataset = csvData.map(_parse_csv_row).batch(64)

In [19]:
# print to view  data set now 
# now the names correspond
print(list(dataset.take(1)))

[({'square_feet': <tf.Tensor: id=132, shape=(64,), dtype=int32, numpy=
array([128536,  56137, 155882, 492898,  44784, 130577,  45086, 393530,
       119262, 153999, 193723, 301307, 282152, 193960, 186572, 426472,
        59498, 258491,  52357,  34819, 167055, 122122, 166395, 168696,
        82939,  57811,  51483, 428647, 108392,  46230,  77632,  38173,
       173479, 121146,   4482,  94751,  67377,   4607, 109263, 186840,
        84346,  68030,  93206, 127632, 230747, 305000, 194111,  89858,
        19454,  80438,  14185, 207115, 100813, 122922,  58377, 287594,
       305047, 105167,  47578,  70852,  32256, 157545, 131745,  71936],
      dtype=int32)>, 'air_temperature': <tf.Tensor: id=127, shape=(64,), dtype=float64, numpy=
array([ -0.69388781,  -0.69388781,  -0.69388781,  -0.69388781,
        -0.69388781,  -0.69388781,  -0.69388781,  -0.69388781,
        -0.69388781,  -0.69388781,  -0.69388781,  -0.69388781,
        -0.69388781,  -0.69388781,  -0.69388781,  -0.69388781,
        -0.69

##### Feature Categorization

In [20]:
# categorising ths features using feature columns

In [21]:
# feature columns for primary use
primary_use = tf.feature_column.numeric_column('primary_use', shape=(14,))
# feature columns for site id
site_id = tf.feature_column.numeric_column('site_id', shape=(9,))
# month day feature columns
month_day = tf.feature_column.numeric_column('month_day', shape=(2,))

In [22]:
# the columns 1 to 4 are numeric -- square feet, air temp, dew temp
numeric_columns = [tf.feature_column.numeric_column(feat) for feat in col_names_[1:4]]

In [23]:
# bring together the feature columns defined above
columns = numeric_columns + [month_day,primary_use,site_id]

In [24]:
# now make a keras  layer from the feature columns
feature_layer = tf.keras.layers.DenseFeatures(columns)

# Build Model

In [25]:
# build a model layer 
model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(4,activation='relu'),
    tf.keras.layers.Dense(2),
    tf.keras.layers.Dense(1, activation='linear')
])

In [36]:
# compile model
model.compile(optimizer =tf.train.AdamOptimizer(),
             loss='mse',
             metrics=['accuracy'])

In [37]:
train_epochs = 3 # increase this to train data better
# for loop trains incrementally
# this is discussed here https://github.com/keras-team/keras/issues/4446
for i in range(train_epochs):    
        model.fit(dataset,epochs=2,steps_per_epoch=10)

Epoch 1/2
10/10 [==============================] - 0s 48ms/step - loss: 113780.3749 - acc: 0.1953
Epoch 2/2
10/10 [==============================] - 0s 5ms/step - loss: 79129.3159 - acc: 0.0562


## Testing

In [38]:
testdata = tf.data.experimental.CsvDataset('illustration_test.csv', data_types, header=False)

In [39]:
test = testdata.map(_parse_csv_row).batch(64)

In [40]:
loss,accuracy = model.evaluate(test)

     16/Unknown - 0s 20ms/step - loss: 74641.3273 - acc: 0.3020

# Conclusion

This is a completely education series . Not to be used in production.